****
# SQL queries using Python and SQLAlquemy
****
<p style="text-align: right"><i>Jesus Perez Colino<br>First version: September 2015<br>Last revision November 2015</i></p>

## About this notebook: 
Notebook prepared by **Jesus Perez Colino** Version 0.1, First Released: 01/12/2014, Alpha.  

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This notebook (WORK IN PROGRESS) 


- **Python & packages versions** to reproduce the results of this notebook: 

In [2]:
from sys import version 
import sqlalchemy
import IPython
print ' Reproducibility conditions for this notebook '.center(90,'-')
print 'Python version:     ' + version
print 'SQLalchemy version: ' + sqlalchemy.__version__
print 'IPython version:    ' + IPython.__version__
print '-'*90

---------------------- Reproducibility conditions for this notebook ----------------------
Python version:     2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) 
[GCC 4.2.1 (Apple Inc. build 5577)]
SQLalchemy version: 1.0.9
IPython version:    4.0.0
------------------------------------------------------------------------------------------


# What is SQLAlchemy

**SQLAlchemy** is a library used to interact with a wide variety of databases. It will enables us to create data models and queries in a manner that feels like normal Python classes and statements. Created by Mike Bayer in 2005, it is probably the most common way of working with relational databases in Python. It can be used to connect to most common databases such as Postgres, MySQL, SQLite, Oracle, and many others. 


## Connecting to a Database

To connect to a database, we need to create a **SQLAlchemy engine**. The **SQLAlchemy Engine** creates a common interface to the database to execute SQL statements. It does this by wrapping a pool of database connections and a dialect in a way that they can work together to provide uniform access to the backend database. This will enables our Python code not to worry about the differences between databases or DBAPIs. SQLAlchemy provides a function to create an engine for us given a connection string and optionally some additional keyword arguments. A connection string is a specially formatted string that provides:

- Database type (Postgres, MySQL, etc.)
- Dialect if the default for the database type (Psycopg2, PyMySQL, etc.)
- Optional authentication details (username and password)
- Location of the database (file or hostname of the database server)
- Optional database server port

In [3]:
from sqlalchemy import MetaData
metadata = MetaData()

from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, DateTime
from datetime import datetime

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:')


## Creating a database

**Table** objects are initialized in SQLAlchemy Core in a supplied **MetaData object** by calling the **Table constructor** with the table name, metadata, and any additional arguments are assumed to be column objects. 

**Column** objects represent each field in the table. The columns are constructed by calling **Column** with a name, type, and then arguments that represent any additional SQL constructs and constraints. **Columns** define the fields that exists in our tables, and they provide the primary means by which we define other constraints through their keyword arguments.

In [4]:
users = Table('users', metadata, 
              Column('user_id', Integer(), primary_key=True), # table’s primary key
              Column('username', String(15), nullable=False, unique=True, index = True),
              Column('email_address', String(255), nullable=False),
              Column('phone', String(20), nullable=False),
              Column('password', String(25), nullable=False),
              Column('created_on', DateTime(), default=datetime.now),
              Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
             )

In [5]:
engine = create_engine('sqlite:///:memory:')

**Metadata** as a kind of catalog of **Table** objects with optional information about the engine and the connection. Those tables can be accessed via a dictionary, `MetaData.tables`. 

In [6]:
metadata.create_all(engine)
connection = engine.connect()

**Keys and constraints** are used as a way to ensure that our data meets certain requirements prior to being stored in the database. The objects that represent keys and constraints can be found inside the base SQLAlchemy module, and three of the more common ones can be imported as shown here.

In [7]:
from sqlalchemy import PrimaryKeyConstraint, UniqueConstraint, CheckConstraint

PrimaryKeyConstraint('user_id', name='user_pk')
UniqueConstraint('username', name='uix_username')
CheckConstraint('unit_cost >= 0.00', name='unit_cost_positive')


CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x1065a89d0>, name='unit_cost_positive')

In [9]:
from sqlalchemy import select, insert
from sqlalchemy.exc import IntegrityError
ins = insert(users).values(
    username="me",
    email_address="me@me.com",
    phone="111-111-1111",
    password="password_me"
)
try:
    result = connection.execute(ins)
except IntegrityError as error:
    print(error.orig.message, error.params)


## Quering a database

To begin building a **query**, we start by using the `select()` function that is very analogous to the standard **SQL** `SELECT` statement:

In [24]:
s = users.select()
results_1 = connection.execute(s)
for result in results_1:
    print result
    print '-'*50
    print(result.username)
    print '-'*50
    print(result.password)
    

(1, u'me', u'me@me.com', u'111-111-1111', u'password_me', datetime.datetime(2015, 11, 15, 12, 46, 27, 221513), datetime.datetime(2015, 11, 15, 12, 46, 27, 221523))
--------------------------------------------------
me
--------------------------------------------------
password_me


Additionally we can query using raw SQL statements or use raw SQL:

In [28]:
result = connection.execute("select * from users").fetchall()
print result

[(1, u'me', u'me@me.com', u'111-111-1111', u'password_me', u'2015-11-15 12:46:27.221513', u'2015-11-15 12:46:27.221523')]


In [35]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key=True)
    username = Column(String(15), nullable=False, unique=True)
    email_address = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now)
    updated_on = Column(DateTime(), default=datetime.now, onupdate=datetime.now)

In [36]:
User.__table__

Table('users', MetaData(bind=None), Column('user_id', Integer(), table=<users>, primary_key=True, nullable=False), Column('username', String(length=15), table=<users>, nullable=False), Column('email_address', String(length=255), table=<users>, nullable=False), Column('phone', String(length=20), table=<users>, nullable=False), Column('password', String(length=25), table=<users>, nullable=False), Column('created_on', DateTime(), table=<users>, default=ColumnDefault(<function <lambda> at 0x10674e050>)), Column('updated_on', DateTime(), table=<users>, onupdate=ColumnDefault(<function <lambda> at 0x10674e140>), default=ColumnDefault(<function <lambda> at 0x10674e0c8>)), schema=None)